In [1]:
import cv2
import mediapipe as mp
import pyautogui
import time
import numpy as np

# 캠 불러오기
cam = cv2.VideoCapture(0)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('eye_tracking.avi', fourcc, 20.0, (640,480))

# 미디어파이프 중 facemesh 사용
# facemesh : 얼굴을 인식하고 얼굴 랜드마크를 추출하는 클래스
# refine_landmarks=True을 통해 랜드마크 좌표를 보다 정확하게 예측할 수 있도록 보정 
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

screen_w, screen_h = 1920, 1080
#pyautogui.size()   # 1920 x 1080

last_record_time = time.time()  # 마지막으로 레코드한 시간


while True:

    # cam.read()에서 반환되는 ret 무시
    # ret : 카메라가 프레임을 읽을 수 있는지에 대한 여부
    # frame : 실제 프레임
    _, frame = cam.read()
    
    # flip 함수를 사용하여 화면을 좌우로 뒤집음(거울과 같은 효과를 위함)
    frame = cv2.flip(frame, 1)
    
    # 프레임을 가져와 BGR에서 RGB로 색상 전환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # 인자로 받은 RGB 이미지로부터 얼굴 랜드마크 정보를 반환 
    output = face_mesh.process(rgb_frame)
    # output.multi_face_landmarks는 모든 인식된 얼굴의 랜드마크를 담은 리스트
    landmark_points = output.multi_face_landmarks
    
    frame_h, frame_w, _ = frame.shape # frame : 480x640x3
    
    if landmark_points:
        # 첫번째 얼굴에 대한 랜드마크 좌표 리스트
        landmarks = landmark_points[0].landmark
        
        # 많은 랜드마크 중 4개의 오른쪽 눈 랜드마크만을 표시
        for id, landmark in enumerate(landmarks[474:478]):
            
            # landmark.x, landmark.y는 0~1 픽셀값이므로 프레임의 높이와 너비를 곱해 프레임에 맞는 좌표를 출력
            # 얻은 좌표는 소수이기 때문에 정수로 변환
            x = int(landmark.x * frame_w)  
            y = int(landmark.y * frame_h)  
            # 프레임 안에 (x,y)를 중심으로 하는 반지름 3의 원을 초록색 동그라미로 표시
            cv2.circle(frame, (x, y), 3, (0, 255, 0))
            
            # 랜드마크 중 하나를 id로 사용, 마우스 커서 이동
            if id == 0: # = landmarks[474]
                
                # 전체 스크린 사이즈로 마우스 영역을 넓힘
                screen_x = screen_w / frame_w * x - 150 # 중앙을 표시하기 위한 조정
                screen_y = screen_h / frame_h* y
                pyautogui.moveTo(screen_x, screen_y)                  
    
    cv2.namedWindow('Eye Tracking', cv2.WINDOW_NORMAL)
    cv2.imshow('Eye Tracking', frame)
    out.write(frame)
    
    
    # 일정 시간마다 현재 좌표값을 파일에 저장
    if time.time() - last_record_time >= 0.3:
        with open('cursor_position.txt', 'a') as f:
            f.write(f'({int(screen_x)}, {int(screen_y)})\n')
        last_record_time = time.time()  # 마지막으로 레코드한 시간 갱신  
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
## 그래프        
img = 255 * np.ones((1080, 1920, 3), dtype=np.uint8)

cv2.line(img, (0, img.shape[0]//3), (img.shape[1], img.shape[0]//3), (0, 0, 0), thickness=1)
cv2.line(img, (0, img.shape[0]//3*2), (img.shape[1], img.shape[0]//3*2), (0, 0, 0), thickness=1)
cv2.line(img, (img.shape[1]//3, 0), (img.shape[1]//3, img.shape[0]), (0, 0, 0), thickness=1)
cv2.line(img, (img.shape[1]//3*2, 0), (img.shape[1]//3*2, img.shape[0]), (0, 0, 0), thickness=1)

coordinates = []

with open("C:/Users/ted43/캡스톤/cursor_position.txt", 'r') as f:
    for line in f:
        x, y = map(int, line.strip()[1:-1].split(','))
        coordinates.append((x, y))
        cv2.circle(img, (x, y), 10, (255, 0, 128), -1)

dense_center = None
dense_count = 0

for i in range(len(coordinates)):
    count = 0
    for j in range(len(coordinates)):
        if i != j:
            dist = np.sqrt((coordinates[i][0]-coordinates[j][0])**2 + (coordinates[i][1]-coordinates[j][1])**2)
            if dist < 100:
                count += 1
    if count > dense_count:
        dense_count = count
        dense_center = coordinates[i]

if dense_center is not None:
    cv2.circle(img, dense_center, 90, (0, 0, 255), thickness=3)

cv2.imshow('Eye Coordinates', img)
cv2.waitKey(0)
cv2.imwrite('img.jpg', img)
cv2.destroyAllWindows()